
# Estimate Focal Length from Orthogonal Vanishing Points

This notebook helps you **compute the camera focal length (in pixels)** from an image, using two **orthogonal directions** in 3D inferred from the image's vanishing points.

### What you'll do
1. Load your image.
2. Click **8 points** total:
   - First 2 points: **Line A1** (direction X)
   - Next 2 points: **Line A2** (same 3D direction as A1; these two lines are parallel in 3D)
   - Next 2 points: **Line B1** (direction Y, **orthogonal** to X in 3D)
   - Final 2 points: **Line B2** (same 3D direction as B1; parallel to B1 in 3D)
3. We compute the two vanishing points (one for each direction) and the focal length via:
   \[ f^2 = -(x_1 - x_c)(x_2 - x_c) - (y_1 - y_c)(y_2 - y_c) \]
   where \((x_c, y_c)\) is the image's optical center (principal point). By default we use the **image center**, or you can override it.

> Tip: If you already know (or can click) the **two vanishing points directly**, skip the clicking workflow and use the **Direct Input** cell below.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from dataclasses import dataclass
from typing import Tuple
from PIL import Image

# Utility: homogeneous cross-product for lines/intersections
def cross(u: np.ndarray, v: np.ndarray) -> np.ndarray:
    return np.cross(u, v)

def to_homog_point(xy: Tuple[float, float]) -> np.ndarray:
    x, y = xy
    return np.array([x, y, 1.0], dtype=float)

def line_from_points(p1: Tuple[float, float], p2: Tuple[float, float]) -> np.ndarray:
    'Return homogeneous line l passing through two image points p1, p2 (both (x,y)).'
    P1 = to_homog_point(p1)
    P2 = to_homog_point(p2)
    l = cross(P1, P2)
    # Normalize by first two components to reduce scale issues (not required mathematically)
    n = np.linalg.norm(l[:2])
    if n > 0:
        l = l / n
    return l

def intersect_lines(l1: np.ndarray, l2: np.ndarray) -> np.ndarray:
    '''Return homogeneous intersection v of lines l1 and l2.'''
    v = cross(l1, l2)
    return v

def to_inhomog_point(v: np.ndarray) -> Tuple[float, float]:
    '''Convert homogeneous point to inhomogeneous pixel coords.'''
    if abs(v[2]) < 1e-12:
        # Point at infinity; return very large coords for plotting/debug
        return (np.inf, np.inf)
    return (float(v[0]/v[2]), float(v[1]/v[2]))

def focal_from_vps(vp1: Tuple[float, float], vp2: Tuple[float, float], cx: float, cy: float) -> float:
    x1, y1 = vp1
    x2, y2 = vp2
    f2 = - (x1 - cx)*(x2 - cx) - (y1 - cy)*(y2 - cy)
    return float(np.sqrt(f2)) if f2 > 0 else np.nan

def draw_line(ax, p1, p2, **kwargs):
    ax.add_line(Line2D([p1[0], p2[0]], [p1[1], p2[1]], **kwargs))



## 1) Load your image
Update `image_path` to point to your file (local path). The image will display for clicking.


In [ ]:

# Change this to your image path, e.g., "my_photo.jpg"
image_path = "your_image.jpg"  # <-- PUT YOUR IMAGE HERE

img = Image.open(image_path).convert("RGB")
w, h = img.size
print(f"Loaded image: {image_path}  (w={w}, h={h})")

fig, ax = plt.subplots(figsize=(8, 8*h/w if w>0 else 6))
ax.imshow(img)
ax.set_title("Loaded image")
ax.set_axis_off()
plt.show()



## 2) Click 8 points to define 4 image lines
- **Click order matters**:
  1. Two clicks: endpoints of **Line A1** (direction X)
  2. Two clicks: endpoints of **Line A2** (parallel to A1 in 3D)
  3. Two clicks: endpoints of **Line B1** (direction Y, orthogonal to X in 3D)
  4. Two clicks: endpoints of **Line B2** (parallel to B1 in 3D)

After clicking 8 points in total, the cell will compute **vanishing points** and **focal length**.


In [ ]:

# Principal point (optical center). Default: image center; change if you know a better estimate.
cx, cy = w/2.0, h/2.0
print(f"Using principal point (cx, cy) = ({cx:.2f}, {cy:.2f})")

fig, ax = plt.subplots(figsize=(8, 8*h/w if w>0 else 6))
ax.imshow(img)
ax.set_title("Click 8 points: A1(2pts), A2(2pts), B1(2pts), B2(2pts)")
ax.set_axis_off()
plt.tight_layout()

# ginput returns a list of (x, y) tuples in pixel coordinates
clicked = plt.ginput(n=8, timeout=0)
plt.close(fig)

if len(clicked) != 8:
    raise RuntimeError(f"Expected 8 points, got {len(clicked)}. Re-run this cell and click again.")

A1_p1, A1_p2, A2_p1, A2_p2, B1_p1, B1_p2, B2_p1, B2_p2 = clicked

# Build lines
lA1 = line_from_points(A1_p1, A1_p2)
lA2 = line_from_points(A2_p1, A2_p2)
lB1 = line_from_points(B1_p1, B1_p2)
lB2 = line_from_points(B2_p1, B2_p2)

# Intersections = vanishing points
vA = intersect_lines(lA1, lA2)  # homogeneous
vB = intersect_lines(lB1, lB2)

vpA = to_inhomog_point(vA)  # (x,y) or (inf,inf)
vpB = to_inhomog_point(vB)

print("Vanishing point A:", vpA)
print("Vanishing point B:", vpB)

# Compute focal length (pixels)
f = focal_from_vps(vpA, vpB, cx, cy)
if np.isnan(f):
    print("Warning: f^2 <= 0 from your clicks. The geometry may be noisy or not perfectly orthogonal.")
else:
    print(f"Estimated focal length f ≈ {f:.3f} pixels")

# Visualization
fig, ax = plt.subplots(figsize=(8, 8*h/w if w>0 else 6))
ax.imshow(img)
ax.set_title("Lines, vanishing points, and principal point")

# Draw clicked lines
draw_line(ax, A1_p1, A1_p2, linewidth=2)
draw_line(ax, A2_p1, A2_p2, linewidth=2)
draw_line(ax, B1_p1, B1_p2, linewidth=2)
draw_line(ax, B2_p1, B2_p2, linewidth=2)

# Draw vanishing points
if np.isfinite(vpA[0]) and np.isfinite(vpA[1]):
    ax.plot(vpA[0], vpA[1], marker='o', markersize=8)
    ax.text(vpA[0]+5, vpA[1]+5, "VP A")

if np.isfinite(vpB[0]) and np.isfinite(vpB[1]):
    ax.plot(vpB[0], vpB[1], marker='o', markersize=8)
    ax.text(vpB[0]+5, vpB[1]+5, "VP B")

# Draw principal point
ax.plot(cx, cy, marker='x', markersize=10)
ax.text(cx+5, cy+5, "Principal point")

ax.set_axis_off()
plt.tight_layout()
plt.show()



## (Optional) Direct Input: enter vanishing points by hand
If you already know \((x_1,y_1)\) and \((x_2,y_2)\) for the two orthogonal vanishing points:


In [ ]:

# Fill these in directly if you prefer
vp1 = None  # e.g., (1234.5, 678.9)
vp2 = None  # e.g., (2500.0, 710.0)

# Optionally override principal point:
# cx, cy = w/2.0, h/2.0

if isinstance(vp1, tuple) and isinstance(vp2, tuple):
    f_direct = focal_from_vps(vp1, vp2, cx, cy)
    if np.isnan(f_direct):
        print("Warning: f^2 <= 0 from your vanishing points.")
    else:
        print(f"[Direct] Estimated focal length f ≈ {f_direct:.3f} pixels")
else:
    print("Set vp1 and vp2 to (x,y) tuples to use direct mode.")
